In [1]:
import local_seqtools.general_utils as tools
import pandas as pd
import numpy as np
import local_conservation_analysis_pipeline.group_conservation_objects as group_tools
import json
import os
import re
import sys
from pathlib import Path
from Bio import AlignIO, Seq, SeqIO, Align

import matplotlib.pyplot as plt
plt.style.use('custom_standard')
# plt.style.use('custom_small')
import seaborn as sns
# pd.options.plotting.backend = "plotly"
from ast import literal_eval

# %load_ext autoreload
# %autoreload 2

In [2]:
zscores = pd.read_csv('./z_scores_wideform.csv')
score_key = pd.read_csv('./z_scores_score_key.csv')
score_indexes=[i for i in zscores.columns if i in score_key['score_index'].unique()]

for col in score_indexes:
    zscores[col] = zscores[col].apply(literal_eval)

zscores['mask_array'] = zscores['mask_array'].str.replace(' ', ',')
zscores['mask_array'] = zscores['mask_array'].apply(literal_eval)
zscores['mask_array'] = zscores['mask_array'].apply(np.array)

def extract_important_scores(s, colname):
    scores = np.array(s[colname])
    maskarr = np.array(s["mask_array"])
    mask = maskarr.nonzero()
    return scores[mask]

zscores2=zscores.copy()
for i in score_indexes:
    zscores2[i] = zscores2.apply(lambda x: extract_important_scores(x, i), axis=1)
    zscores2[i] = zscores2[i].apply(np.mean)

# zscores2 = zscores2[zscores2['ELM_motif_class'] != 'TRAF6']
passing_ref_id_list = zscores['reference_index'].unique()

In [3]:
zscores['mask_array'].isnull().sum()

0

# supplemental table - counts for each set for each SLiM in the benchmark

In [4]:
supp_folder = Path('./supplement')
class_counts_folder = supp_folder / 'benchmark_class_counts'
class_counts_folder.mkdir(exist_ok=True, parents=True)
# zscores.groupby('verified interaction')['ELM_motif_class'].value_counts().unstack().to_csv(class_counts_folder / 'benchmark_class_counts.csv')
# zscores.groupby('verified interaction')['ELM_motif_class'].value_counts().unstack()
temp = zscores.groupby('verified interaction')['ELM_motif_class'].value_counts().unstack().T.copy()
re_map = zscores.groupby('ELM_motif_class').first()['regex'].to_dict()
mask_map = zscores.groupby('ELM_motif_class').first()['mask_array'].to_dict()
mask_map['LIG_14-3-3_CanoR_1'] = 'No mask'
temp['regex'] = temp.index.map(re_map)
temp['mask_array'] = temp.index.map(mask_map)
temp.to_csv(class_counts_folder / 'benchmark_class_counts.csv')
temp

verified interaction,False,True,regex,mask_array
ELM_motif_class,,,,
DOC_WW_Pin1_4,227,42,...([ST])P.,"[0, 0, 0, 1, 1, 0]"
Ena/VASP ([FL]PPPP),249,34,[FL]PPPP,"[1, 1, 0, 1, 1]"
LIG_14-3-3_CanoR_1,255,41,"R[^DE]{0,2}[^DEPG]([ST])(([FWYLMV].)|([^PRIKGN...",No mask
LIG_AP2alpha_2,249,49,DP[FW],"[1, 1, 1]"
LIG_EH_1,269,49,.NPF.,"[0, 1, 1, 1, 0]"
LIG_SH2_GRB2like,271,18,(Y)([EDST]|[MLIVAFYHQW])N.,"[1, 1, 1, 0]"
LIG_SH3_CIN85_PxpxPR_1,252,28,P.[AP].PR,"[1, 0, 1, 0, 1, 1]"
TRAF6,221,7,...P.E..[FYWDE],"[0, 0, 0, 1, 0, 1, 0, 0, 1]"


# supplementary table - conservation scores

In [5]:
score_indexes

['0.0',
 '2.0',
 '4.0',
 '8.0',
 '9.0',
 '18.0',
 '24.0',
 '28.0',
 '29.0',
 '32.0',
 '33.0',
 '36.0',
 '37.0',
 '40.0',
 '41.0',
 'kibby_hit_zscores']

In [6]:
score_key[(score_key['lflank']!='5.0')]

,score_index,aln_type,level,score_key,table_file,lflank,rflank,columnwise_score_function_name,num_passing
0,0.0,MSA - MAFFT,Tetrapoda,aln_property_entropy,/home/jch/Documents/08-benchmark/benchmark/ben...,MSA,MSA,property_entropy,2490
1,1.0,MSA - MAFFT,Tetrapoda,aln_shannon_entropy,/home/jch/Documents/08-benchmark/benchmark/ben...,MSA,MSA,shannon_entropy,2490
2,2.0,MSA - MAFFT,Vertebrata,aln_property_entropy,/home/jch/Documents/08-benchmark/benchmark/ben...,MSA,MSA,property_entropy,2644
3,3.0,MSA - MAFFT,Vertebrata,aln_shannon_entropy,/home/jch/Documents/08-benchmark/benchmark/ben...,MSA,MSA,shannon_entropy,2644
4,4.0,MSA - MAFFT,Metazoa,aln_property_entropy,/home/jch/Documents/08-benchmark/benchmark/ben...,MSA,MSA,property_entropy,2474
5,5.0,MSA - MAFFT,Metazoa,aln_shannon_entropy,/home/jch/Documents/08-benchmark/benchmark/ben...,MSA,MSA,shannon_entropy,2474
7,7.0,Pairwise,Tetrapoda,pairk_aln_lf0_rf0_edssmat50,/home/jch/Documents/08-benchmark/benchmark/ben...,0.0,0.0,shannon_entropy,2652
9,9.0,Pairwise,Tetrapoda,pairk_aln_lf0_rf0_edssmat50,/home/jch/Documents/08-benchmark/benchmark/ben...,0.0,0.0,property_entropy,2652
12,12.0,Pairwise,Vertebrata,pairk_aln_lf0_rf0_blosum62,/home/jch/Documents/08-benchmark/benchmark/ben...,0.0,0.0,shannon_entropy,2822
13,13.0,Pairwise,Vertebrata,pairk_aln_lf0_rf0_grantham,/home/jch/Documents/08-benchmark/benchmark/ben...,0.0,0.0,shannon_entropy,2822


In [54]:
# score_key_filt = score_key[score_key['score_index'].isin(score_indexes)]
# score_key_filt = score_key_filt[score_key_filt['score_index']!= 'kibby_hit_zscores']

In [13]:
cols2export=list(score_key[
    (~score_key['lflank'].isin(['5.0','10.0','2.0']))&
    (score_key['columnwise_score_function_name']!='shannon_entropy')&
    (~score_key['score_key'].isin(['pairk_aln_lf0_rf0_blosum62','pairk_aln_lf0_rf0_grantham', 'kibby', 'pairk_aln_needleman_lf0_rf0_edssmat50']))
].score_index.unique())
cols2export
score_key_for_rename = score_key[score_key['score_index'].isin(cols2export)].copy()
score_key_for_rename['aln_type'] = score_key_for_rename['aln_type'].replace('Pairwise', 'pairk')
score_key_for_rename['aln_type'] = score_key_for_rename['aln_type'].replace('Pairwise embedding', 'pairk - ESM2 embeddings')

In [15]:
score_key[score_key['score_index'].isin(cols2export)]

,score_index,aln_type,level,score_key,table_file,lflank,rflank,columnwise_score_function_name,num_passing
0,0.0,MSA - MAFFT,Tetrapoda,aln_property_entropy,/home/jch/Documents/08-benchmark/benchmark/ben...,MSA,MSA,property_entropy,2490
2,2.0,MSA - MAFFT,Vertebrata,aln_property_entropy,/home/jch/Documents/08-benchmark/benchmark/ben...,MSA,MSA,property_entropy,2644
4,4.0,MSA - MAFFT,Metazoa,aln_property_entropy,/home/jch/Documents/08-benchmark/benchmark/ben...,MSA,MSA,property_entropy,2474
9,9.0,Pairwise,Tetrapoda,pairk_aln_lf0_rf0_edssmat50,/home/jch/Documents/08-benchmark/benchmark/ben...,0.0,0.0,property_entropy,2652
24,24.0,Pairwise,Vertebrata,pairk_aln_lf0_rf0_edssmat50,/home/jch/Documents/08-benchmark/benchmark/ben...,0.0,0.0,property_entropy,2822
29,29.0,Pairwise,Metazoa,pairk_aln_lf0_rf0_edssmat50,/home/jch/Documents/08-benchmark/benchmark/ben...,0.0,0.0,property_entropy,2844
33,33.0,Pairwise embedding,Tetrapoda,pairk_aln_embedding_lf0_rf0,/home/jch/Documents/08-benchmark/benchmark/ben...,0.0,0.0,property_entropy,2652
37,37.0,Pairwise embedding,Vertebrata,pairk_aln_embedding_lf0_rf0,/home/jch/Documents/08-benchmark/benchmark/ben...,0.0,0.0,property_entropy,2822
41,41.0,Pairwise embedding,Metazoa,pairk_aln_embedding_lf0_rf0,/home/jch/Documents/08-benchmark/benchmark/ben...,0.0,0.0,property_entropy,2844


In [16]:
cols2export

['0.0', '2.0', '4.0', '9.0', '24.0', '29.0', '33.0', '37.0', '41.0']

In [17]:
zscores_export = zscores2[
    [
        'ELM_motif_class',
        'verified interaction',
        'regex',
        'UniprotID',
        'gene_id',
        'hit start position',
        'hit end position',
        'hit_sequence',
        'mask_array',
    ]
    + cols2export
    + ['kibby_hit_zscores']
].copy()
rn_dict = {
    'ELM_motif_class': 'SLiM class',
    'gene_id': 'OrthoDB id',
    'kibby_hit_zscores': 'Kibby',
}
zscores_export = zscores_export.rename(columns=rn_dict)

def score_index_2_col_title(score_index, score_key_df):
    df = score_key_df[score_key_df['score_index']==score_index].copy()
    assert len(df)==1, f"More than one match for {score_index}"
    level=df['level'].values[0]
    aln_type=df['aln_type'].values[0]
    title = f'{aln_type} - {level}'
    return title

rn_dict2 ={}
for i in cols2export:
    rn_dict2[i] = score_index_2_col_title(i, score_key_for_rename)
zscores_export = zscores_export.rename(columns=rn_dict2)
zscores_export

,SLiM class,verified interaction,regex,UniprotID,OrthoDB id,hit start position,hit end position,hit_sequence,mask_array,MSA - MAFFT - Tetrapoda,MSA - MAFFT - Vertebrata,MSA - MAFFT - Metazoa,pairk - Tetrapoda,pairk - Vertebrata,pairk - Metazoa,pairk - ESM2 embeddings - Tetrapoda,pairk - ESM2 embeddings - Vertebrata,pairk - ESM2 embeddings - Metazoa,Kibby
0,LIG_AP2alpha_2,True,DP[FW],O08838,10116_0:004cdf,356,358,DPF,"[1, 1, 1]",1.312759,0.865163,-0.906534,0.765409,0.387204,-0.484739,1.047175,0.779145,-0.120440,1.727165
1,LIG_AP2alpha_2,True,DP[FW],Q05140,10116_0:002e5e,399,401,DPF,"[1, 1, 1]",0.180739,-1.107886,-1.174238,0.284105,0.369749,0.344503,0.547614,0.943751,0.766580,2.232242
2,LIG_AP2alpha_2,True,DP[FW],Q05140,10116_0:002e5e,473,475,DPF,"[1, 1, 1]",-0.091241,1.006945,0.850580,0.284105,0.369749,0.344503,0.547614,0.939865,0.753180,2.818305
3,LIG_AP2alpha_2,True,DP[FW],P98082,9606_0:0016b2,292,294,DPF,"[1, 1, 1]",1.385546,1.510597,0.930359,0.907574,1.022667,1.213079,1.233372,1.392867,1.850673,1.770583
4,LIG_AP2alpha_2,True,DP[FW],P98082,9606_0:0016b2,297,299,DPF,"[1, 1, 1]",1.432202,1.150243,0.472766,0.907574,1.022667,1.213079,1.233372,1.174462,1.721987,1.799091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256,TRAF6,False,...P.E..[FYWDE],O43283,9606_0:000dc7,866,874,SNSPDELAD,"[0, 0, 0, 1, 0, 1, 0, 0, 1]",-0.195058,-1.007573,-1.352457,-0.354896,-0.844113,0.049921,-0.241852,-0.725470,-0.323105,-0.054754
2257,TRAF6,False,...P.E..[FYWDE],A0A5F9ZHS8,9606_0:00386f,1011,1019,IRDPSEFEY,"[0, 0, 0, 1, 0, 1, 0, 0, 1]",0.337852,0.093147,0.100192,0.445612,0.231898,-0.708671,0.356714,0.288645,0.332622,-0.568793
2258,TRAF6,False,...P.E..[FYWDE],P22087,9606_0:004480,109,117,NLVPGESVY,"[0, 0, 0, 1, 0, 1, 0, 0, 1]",0.038985,-0.043496,-0.163402,0.063965,-0.014289,-0.293095,0.463843,0.113719,-0.058505,0.474632
2259,TRAF6,False,...P.E..[FYWDE],NaN,9606_0:00123d,2623,2631,NSSPEEVQF,"[0, 0, 0, 1, 0, 1, 0, 0, 1]",0.776609,-0.973246,-0.257565,0.768418,0.791382,0.967131,0.707588,0.793146,1.063255,0.202799


In [18]:
zscores_export=zscores_export.rename(columns={'hit start position': 'match start position', 'hit end position': 'match end position', 'hit_sequence': 'match sequence'})

In [19]:
output_dir=supp_folder / 'benchmark_zscores'
output_dir.mkdir(exist_ok=True, parents=True)
zscores_export.to_csv(output_dir / 'benchmark_scores_v2.csv', index=False)